In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [2]:
# если у вас линукс / мак / collab или ещё какая-то среда, в которой работает wget, можно так:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2023-01-30 21:31:49--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2023-01-30 21:31:50--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc387223bbfa0a41166ab868aa53.dl.dropboxusercontent.com/cd/0/inline/B1nNil_Y-s_fTg4bdSanHv9UPGNONhJDM1F-C_t4IkbSZAJhhImgo3kw-1iZCle4Q219zyy0W7hFUgFAzKGpZM6GFXRPpX3p0Ae19DrMqn2tTrS-Sp6s6IfXTZ-RQevTI2O1Bli7-aeblcdcl-BvhSRxhaPmcusDQhuCVFRD3Ad1Fw/file# [following]
--2023-01-30 21:31:50--  https://uc387223bbfa0a41166ab868aa53.dl.dropboxusercontent.com/cd/0/inline/B1nNil_Y-s_fTg4bdSanHv9UPGNONhJDM1F-C_t4IkbSZAJhhImgo3kw-1iZCle4Q219zyy0W7hFUgFA

In [3]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [4]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [5]:
df.tail()

,text,label
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
111922,Такси везет меня на работу. Раздумываю приплат...,negative


In [6]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [7]:
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
from nltk import ngrams
import nltk
from nltk import collocations 
nltk.download('genesis')
nltk.download('punkt')
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('russian'))

[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
from string import punctuation
noise = stopwords.words('russian') + list(punctuation)


In [14]:
from collections import Counter

In [15]:
corpus = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]
print(len(corpus))
corpus[:10]

2870536


['first_timee', 'хоть', 'я', 'и', 'школота', 'но', 'поверь', 'у', 'нас', 'то']

In [16]:
freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[('не', 69472),
 ('и', 55166),
 ('в', 52902),
 ('я', 52818),
 ('RT', 38070),
 ('на', 35759),
 ('http', 32998),
 ('что', 31541),
 ('с', 27217),
 ('а', 26860)]

-----

## Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

In [17]:
# Подготовим три группы токенов: с высокой, средней и низкой частотой

high_tokens = set()
med_tokens = set()
lit_tokens = set()
h = 10000 # если частота выше этого значения будем считать что это токен с высокой частотой
m = 500 # если частота ниже этого значения будем считать что это токен с низкой частотой
for i in freq_dict_sorted:
    if i[1] > h:
        high_tokens.add(i[0])
    elif i[1] < m:
        lit_tokens.add(i[0])
    else:
        med_tokens.add(i[0])
print(len(high_tokens),len(med_tokens),len(lit_tokens))                     
                       

29 507 350587


In [18]:
# создадим фильтр (стопслова) с добавлением токенов средней и низкой частоты, 
# то есть будем считать только по токенам с высокой частотой
sw = noise + list(lit_tokens) + list(med_tokens)

In [19]:
import warnings
warnings.filterwarnings("ignore")

In [20]:
%%time
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=sw)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.80      0.57      0.67     39163
    positive       0.42      0.69      0.52     17546

    accuracy                           0.61     56709
   macro avg       0.61      0.63      0.60     56709
weighted avg       0.69      0.61      0.62     56709

CPU times: user 1min 15s, sys: 5.99 s, total: 1min 21s
Wall time: 1min 17s


In [21]:
# посчитаем только по токенам с средней частотой
sw = noise + list(high_tokens) + list(lit_tokens)

In [22]:
%%time
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=sw)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.57      0.68      0.62     23219
    positive       0.74      0.64      0.69     33490

    accuracy                           0.66     56709
   macro avg       0.65      0.66      0.65     56709
weighted avg       0.67      0.66      0.66     56709

CPU times: user 1min 16s, sys: 6.69 s, total: 1min 23s
Wall time: 1min 18s


In [23]:
# посчитаем только по токенам с низкой частотой
sw = noise + list(high_tokens) + list(med_tokens)

In [24]:
%%time
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=sw)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.79      0.72      0.75     30772
    positive       0.70      0.78      0.74     25937

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.74     56709
weighted avg       0.75      0.75      0.75     56709

CPU times: user 57.3 s, sys: 10.8 s, total: 1min 8s
Wall time: 58.8 s


Лучшая точность получилась на токенах с низкой частотностью

Задание 2.

найти фичи с наибольшей значимостью, и вывести их

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
# Оставим знаки пунктуации  
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stopwords.words('russian'))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27910
    positive       1.00      1.00      1.00     28799

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



In [27]:
# похоже что существенный вклад дают симолы которые участвуют в смайлах. Проверим
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [28]:
p = list('"#$%&\'+,.<=>?@[\\]^`{|}~')
p

['"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '+',
 ',',
 '.',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '`',
 '{',
 '|',
 '}',
 '~']

In [29]:
myfilter = stopwords.words('russian') + p
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words= myfilter)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27881
    positive       1.00      1.00      1.00     28828

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



In [30]:
pp = []
for i in punctuation:
    cool_token = i
    pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
    pp.append((i,accuracy_score(pred, y_test)))
print(pp)

[('!', 0.5136398102593944), ('"', 0.5047170643107796), ('#', 0.5054224197217373), ('$', 0.4933255744238128), ('%', 0.4958295861327126), ('&', 0.4927789239803206), ("'", 0.49321977111216914), ('(', 0.027561762683171984), (')', 0.9137526671251477), ('*', 0.5122643672080269), ('+', 0.49364298435874376), (',', 0.5066920594614611), ('-', 0.5120174928141917), ('.', 0.5069212999700224), ('/', 0.5432823714048917), (':', 0.5470912906240633), (';', 0.49664074485531395), ('<', 0.49323740499744306), ('=', 0.4924438801601157), ('>', 0.49323740499744306), ('?', 0.5042233155231092), ('@', 0.5681285157558765), ('[', 0.49313160168579945), ('\\', 0.49321977111216914), (']', 0.49327267276799097), ('^', 0.49979721031934965), ('_', 0.5179601121515104), ('`', 0.4929023611772382), ('{', 0.49323740499744306), ('|', 0.48907580807279266), ('}', 0.49325503888271705), ('~', 0.4930787000299776)]


Выведем фичи с наибольшей значимостью

In [31]:
for i in sorted(pp, key=lambda x: -x[1])[:10]:
    print(i[0],'  -   ' ,i[1])


)   -    0.9137526671251477
@   -    0.5681285157558765
:   -    0.5470912906240633
/   -    0.5432823714048917
_   -    0.5179601121515104
!   -    0.5136398102593944
*   -    0.5122643672080269
-   -    0.5120174928141917
.   -    0.5069212999700224
,   -    0.5066920594614611


----

Задание 3.

1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера

3) убедиться что для сетки нет переобучения

In [32]:
%%time
myfilter = stopwords.words('russian') + p

count_vect = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words= myfilter)
bow = count_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))


              precision    recall  f1-score   support

    negative       1.00      0.56      0.72     50152
    positive       0.23      1.00      0.37      6557

    accuracy                           0.61     56709
   macro avg       0.61      0.78      0.54     56709
weighted avg       0.91      0.61      0.68     56709

CPU times: user 58.9 s, sys: 12.2 s, total: 1min 11s
Wall time: 1min


In [33]:
%%time
myfilter = stopwords.words('russian')
tf_vect = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words= myfilter)
bow = tf_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(tf_vect.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27910
    positive       1.00      1.00      1.00     28799

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709

CPU times: user 56.1 s, sys: 7.56 s, total: 1min 3s
Wall time: 57.2 s


In [34]:
from sklearn.feature_extraction.text import HashingVectorizer

In [35]:
vectorizer = HashingVectorizer(n_features=2**4,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))


              precision    recall  f1-score   support

    negative       0.51      0.54      0.52     26185
    positive       0.58      0.55      0.57     30524

    accuracy                           0.55     56709
   macro avg       0.55      0.55      0.55     56709
weighted avg       0.55      0.55      0.55     56709



In [36]:
vectorizer = HashingVectorizer(n_features=2**8,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))


              precision    recall  f1-score   support

    negative       0.60      0.61      0.60     27300
    positive       0.63      0.62      0.62     29409

    accuracy                           0.61     56709
   macro avg       0.61      0.61      0.61     56709
weighted avg       0.61      0.61      0.61     56709



In [37]:
vectorizer = HashingVectorizer(n_features=2**10,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))


              precision    recall  f1-score   support

    negative       0.64      0.65      0.64     27340
    positive       0.67      0.65      0.66     29369

    accuracy                           0.65     56709
   macro avg       0.65      0.65      0.65     56709
weighted avg       0.65      0.65      0.65     56709



In [38]:
vectorizer = HashingVectorizer(n_features=2**12,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))


              precision    recall  f1-score   support

    negative       0.67      0.69      0.68     27363
    positive       0.70      0.69      0.70     29346

    accuracy                           0.69     56709
   macro avg       0.69      0.69      0.69     56709
weighted avg       0.69      0.69      0.69     56709

